In [ ]:
import numpy as np 
from scipy.special import gammaincinv

from astropy.modeling import models, custom_model
from astropy.table import Table 
from astropy.modeling import models, fitting, powerlaws

from petrofit.models import sersic_enclosed, sersic_enclosed_inv
from petrofit.models import petrosian_profile, sersic_enclosed
from petrofit.petrosian import Petrosian, PetrosianCorrection
from petrofit.models import make_grid
from petrofit.utils import closest_value_index, get_interpolated_values

%matplotlib inline

from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = [10, 10]
plt.rcParams['image.origin'] = 'lower'
#plt.rcParams.update({'font.size': 17})


import matplotlib.pyplot as plt

SMALL_SIZE = 15
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

#plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


In [ ]:
def savefig(filename):
    plt.savefig(filename, dpi=70, bbox_inches = 'tight', pad_inches = 0.2)

# Figure 1: Effective Radius and Intensity ($r_e$  and $I_{e}$)

In [ ]:
r_eff=25
n = 1 
model = models.Sersic1D(r_eff=r_eff, n=n)

r_list = np.arange(0, 101, 1)

x_array = r_list/r_eff
plt.plot(x_array, model(r_list), c='b', linewidth=4, label="Sersic Profile with n={}".format(n))
plt.plot(-x_array, model(r_list), c='b', linewidth=4)

alpha = 1

plt.fill_between(x_array,model(r_list), color='honeydew', hatch='/', edgecolor='green', alpha=alpha)
plt.fill_between(-x_array,model(r_list), color='honeydew', hatch='/', edgecolor='green', alpha=alpha)

r_eff_list = np.arange(0, r_eff+1, 1)
plt.fill_between(r_eff_list/r_eff,model(r_eff_list), color='aliceblue', hatch='.', edgecolor='blue', alpha=alpha)
plt.fill_between(-r_eff_list/r_eff,model(r_eff_list), color='aliceblue', hatch='.', edgecolor='blue', alpha=alpha)


plt.axvline(1, c='black', label='r = r_e ({} and -{})'.format(r_eff, r_eff), linestyle='--')
plt.axvline(-1, c='black', linestyle='--')
plt.axvline(c='black')

plt.axhline(1, c='black', linestyle='--', label='I_e = {}'.format(model.amplitude.value))


plt.title("Sérsic Profile")
plt.xlabel("r / $r_{e}$")
plt.ylabel("I(r)")
plt.legend()
plt.ylim(0, None)
#plt.xlim(-100, 100)


# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker
# import numpy as np

# @ticker.FuncFormatter
# def major_formatter(x, pos):
#     label = str(int(abs(x)))
#     return label

# ax = plt.gca()

# ax.xaxis.set_major_formatter(major_formatter)

savefig("sersic_radii.png")
plt.show()



# Figure 2: Curve of Growth

In [ ]:
r_list = np.arange(0, 201, 1)

amplitude, r_eff, n = [1, 25, 1]

flux_list = sersic_enclosed(r_list, amplitude, r_eff, n)
total_flux = sersic_enclosed(np.inf, amplitude, r_eff, n)


x_array = r_list/r_eff

f, ax = plt.subplots(1,2, figsize=[20,10])
for i in range(2):
    plt.sca(ax[i])
    plt.plot(x_array, flux_list, c='Blue', linewidth=3, label='Curve of Growth')

    plt.axhline(total_flux, c='black', linestyle='-', linewidth=3, label='Total Flux')
    
    plt.axhline(total_flux/2, c='black', linestyle='--', linewidth=2,)
    plt.axvline(1, c='black', linestyle='--', linewidth=2, label='Half Total Flux and Half Light Radius $r_e = {}$'.format(r_eff))

    eff = flux_list.copy()
    eff[r_eff:] = total_flux/2
    

    if i == 1:
        alpha = 0.75
        plt.fill_between(x_array, np.zeros_like(eff)+eff[-1], color='aliceblue', hatch='.', edgecolor='blue', alpha=alpha)
        plt.fill_between(x_array, np.zeros_like(eff)+flux_list[-1], np.zeros_like(eff)+eff[-1], color='honeydew', hatch='/', edgecolor='green', alpha=alpha)



    leg = plt.legend()
#     for lh in leg.legendHandles: 
#         lh._legmarker.set_alpha(1)
#         lh.set_alpha(1)

    plt.title("Curve of Growth (COG)")
    plt.xlabel("r / $r_{e}$")
    plt.ylabel("L($\leq$r)")
    legend = plt.legend()
    legend.get_frame().set_alpha(None)
    plt.ylim(0, None)
    plt.xlim(0, max(x_array))


savefig("sersic_cog.png")
plt.show()

In [ ]:
r_list = np.arange(0, 201, 1)

amplitude, r_eff, n = [1, 25, 1]

flux_list = sersic_enclosed(r_list, amplitude, r_eff, n)
total_flux = sersic_enclosed(np.inf, amplitude, r_eff, n)
x_array = r_list/r_eff

i = 1
plt.plot(x_array, flux_list, c='blue', linewidth=3, label='Curve of Growth')

plt.axhline(total_flux, c='black', linestyle='-.', linewidth=3, label='Total Flux')

plt.axhline(total_flux/2, c='black', linestyle='--', linewidth=2,)
plt.axvline(1, c='black', linestyle='--', linewidth=2, label='Half Total Flux and Half Light Radius $r_e = {}$'.format(r_eff))

eff = flux_list.copy()
eff[r_eff:] = total_flux/2

if i == 1:
    alpha = 0.5
    plt.fill_between(x_array, np.zeros_like(eff)+eff[-1], color='aliceblue', hatch='.', edgecolor='blue', alpha=alpha)
    plt.fill_between(x_array, np.zeros_like(eff)+flux_list[-1], np.zeros_like(eff)+eff[-1], color='honeydew', hatch='/', edgecolor='green', alpha=alpha)


leg = plt.legend()
# for lh in leg.legendHandles: 
#     lh._legmarker.set_alpha(1)
#     lh.set_alpha(1)

plt.title("Sérsic Curve of Growth (COG)")
plt.xlabel("r / $r_{e}$")
plt.ylabel("L($\leq$r)")
legend = plt.legend()
legend.get_frame().set_alpha(None)
plt.ylim(0, None)
plt.xlim(0, max(x_array))



#savefig("sersic_single_cog.png")
plt.show()

# Sersic Index ($n$)

In [ ]:
r_eff=25
n = 1 


r_list = np.arange(0, 101, 1)


for line_style, c, n in zip(['-','-','-'], ['g', 'b', 'purple'],[0.5, 1, 4]):
    
    b_n = gammaincinv(2. * n, 0.5)
    model = models.Sersic1D(amplitude=1/np.exp(b_n),  r_eff=r_eff, n=n)
    
    plt.plot(r_list, model(r_list), c=c, linewidth=2, label="n = {}".format(n), linestyle=line_style)
    plt.plot(-r_list, model(r_list), c=c, linewidth=2, linestyle=line_style)


plt.title("Sérsic Indices")
plt.xlabel("Radius r")
plt.ylabel("I(r)")
plt.legend()
plt.ylim(0, None)
plt.xlim(-100, 100)

#savefig("sersic_index.png")
plt.show()


# Figure 3: Petrosian Profiles 

In [ ]:
r_list = np.arange(0, 201, 1)

amplitude, r_eff, n = [1, 25, 1]

x_array = r_list/r_eff

petro_list = petrosian_profile(r_list, r_eff, n)
petro_list[0] = 1

plt.plot(x_array, petro_list, color="b", linewidth=4, label='Petrosian Profile')


plt.title("Petrosian Profile")
plt.xlabel("r / $r_{e}$")
plt.ylabel("Petrosian Value")
plt.legend()
plt.ylim(0, 1)
plt.xlim(0, max(x_array))


#savefig("sersic_petrosian.png")


plt.show()
flux_list = sersic_enclosed(r_list, amplitude, r_eff, n)
area_list = np.pi * r_list**2 

p = Petrosian(r_list[1:], area_list[1:], flux_list[1:],)

p.plot()

plt.title("Petrosian Profile")
plt.xlabel("Radius r")
plt.ylabel("Petrosian Value")
plt.legend()
plt.ylim(0, 1)
plt.xlim(0, max(r_list))

#savefig("sersic_petrosian_eta.png")



# Figure 4 and 5: Petrosian Corrections 

In [ ]:
# Fitted models 
c_relation = models.Polynomial1D(6, c0=2.26194802, c1=-3.61130833, c2=3.8219758, c3=-1.64146601, c4=0.38059409, c5=-0.0450384, c6=0.00221922)
c2080_to_n = models.Polynomial1D(5, c0=-0.41844073, c1=0.20487513, c2=0.08626531, c3=0.01067072, c4=-0.00082523, c5=0.00002486)
n_to_epsilon = models.Polynomial1D(5, c0=-6.54870813, c1=-2.15040843, c2=-0.28993623, c3=-0.04099376, c4=-0.00046837, c5=-0.00022305) + models.Exponential1D(amplitude=7.48787292, tau=2.6876055)


In [ ]:
# pc = PetrosianCorrection("plot_correction_gid_no_psf.yaml")

# keyx = 'c_index'
# keyy = 'epsilon'

# x = []
# y = []

# for i, key in enumerate(list(pc.grid.keys())[:]):
#     if key < 10 or key > 90:
#         continue
#     a = (i+2) / len(pc.grid)
#     data = pc.grid[key]
    
#     key_data_x = np.array(data[keyx])
#     key_data_y = np.array(data[keyy])
    
# #     idx = np.where(key_data_x < 4)
# #     key_data_x = key_data_x[idx]
# #     key_data_y = key_data_y[idx]
    
#     plt.scatter(key_data_x, key_data_y, c='black', alpha=a, label='r_eff={}'.format(key))

#     x += list(key_data_x)
#     y += list(key_data_y)

# x = np.array(x)
# y = np.array(y)

# # # initialize a linear fitter
# fit = fitting.LevMarLSQFitter()
# # # initialize a linear model

# line_init = models.Exponential1D() * powerlaws.BrokenPowerLaw1D() * models.Polynomial1D(5)
# #fitted_line = fit(line_init, x, y, maxiter=10000, acc=1e-07,)

# #fitted_line = fit(n_to_epsilon, c2080_to_n(c_relation(x)), y, maxiter=1000000, acc=1e-07,)

# fitted_line = lambda x: n_to_epsilon(c2080_to_n(c_relation(x)))

# key_red = 50
# #data_red = pc.grid[key_red]
# #red_x, red_y = get_interpolated_values(key_data_x, key_data_y)

# red_x = np.arange(x.min(), 4.77, 0.01)
# #plt.plot(red_x, red_y,  c='red', label='Reference (No PSF r_eff={})'.format(key_red))

# plt.plot(red_x, fitted_line(red_x), c='red', label='Reference Line')
# plt.scatter(red_x[-1], fitted_line(red_x[-1]), c='red')

# plt.title("Relationship b/w Concentration and Epsilon")
# plt.xlabel("C2080")
# plt.ylabel("Epsilon")
# plt.legend()

# savefig("concentration_index_vs_epsilon.png")

# plt.show()
# #########################################################

# pc = PetrosianCorrection("plot_correction_gid.yaml")

# x = []
# y = []

# for i, key in enumerate(list(pc.grid.keys())[:]):
#     if key < 10 or key > 90:
#         continue
#     a = (i+2) / len(pc.grid)
#     data = pc.grid[key]
#     key_data_x = np.array(data[keyx])
#     key_data_y = np.array(data[keyy])
    
# #     idx = np.where(key_data_x < 4)
# #     key_data_x = key_data_x[idx]
# #     key_data_y = key_data_y[idx]
    
#     plt.scatter(key_data_x, key_data_y, c='black', alpha=a, label='r_eff={}'.format(key))

#     x += list(key_data_x)
#     y += list(key_data_y)

# x = np.array(x)
# y = np.array(y)

# plt.plot(red_x, fitted_line(red_x), c='red', label='Reference Line')
# plt.scatter(red_x[-1], fitted_line(red_x[-1]), c='red')
# #plt.plot(red_x, red_y,  c='red', label='Reference (No PSF r_eff={})'.format(key_red))

# plt.title("F105W Relationship b/w Concentration and Epsilon")
# plt.xlabel("C2080")
# plt.ylabel("Epsilon")
# plt.legend()

# #savefig("f105w_concentration_index_vs_epsilon.png")

# print("range_c2080 = ({:.2f}, {:.2f})".format(x.min(), x.max()))

# plt.show()
    

# Figure 15: Fraction of total flux recovered 

In [ ]:
# from astropy.table import Table

# t_raw = Table.read('fine_petro_table_no_psf.csv')

# t = t_raw[np.where(t_raw['sersic_n'] <=15)].copy()


In [ ]:
# plt.scatter(t['sersic_n'], t['uncorr_total_flux']/t['corr_total_flux'], c='black')

# plt.axvline(4, linestyle='--', label='(n=4, fraction=0.838)', c='b')
# plt.axhline(0.838, linestyle='--', c='b')

# plt.xlabel('Sérsic Index n')
# plt.ylabel('Fraction of total flux recoved')
# plt.title('Fraction of total flux recoved by $\epsilon = 2$')
# plt.legend()
# #|plt.axhline(1)

# Figure 6: Oversample grids 

In [ ]:
# size = 4
# x, y = make_grid(size, 1)
# plt.scatter(x, y, label="One to one sampling", s=200)

# x, y = make_grid(size, 2)
# plt.scatter(x, y, label="Oversampling by a factor of 2", s=75, marker='s')

# x, y = make_grid(size, 4)
# plt.scatter(x, y, label="Oversampling by a factor of 4", s=30, marker='x')


# plt.title("Sampling Grids")
# plt.xlabel("X Axis")
# plt.ylabel("Y Axis")


# ax = plt.gca()

# # Major ticks every 20, minor ticks every 5
# major_ticks = np.arange(0, size, 1)
# minor_ticks = np.arange(0, size, 0.5)

# ax.set_xticks(major_ticks)
# ax.set_xticks(minor_ticks, minor=True)
# ax.set_yticks(major_ticks)
# ax.set_yticks(minor_ticks, minor=True)

# ax.grid(which='minor', linewidth=2)
# ax.grid(which='major', linewidth=1, linestyle='--')

# leg = plt.legend(loc='lower right', )
# leg.get_frame().set_alpha(1)
# leg.get_frame().set_edgecolor('black')

# # leg.get_frame().set_facecolor('black')
# # for text in leg.get_texts():
# #     plt.setp(text, color = 'w')

# plt.xlim(-0.5, size-.5)
# plt.ylim(-0.5, size-.5)
# savefig("sampling_grids.png")



# plt.show()